In [126]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

In [86]:
tr=pd.read_csv('train.csv')

In [89]:
import re
import unicodedata
import pandas as pd

# 나이를 추출하는 함수
def extract_age(text):
    pattern = r'\d+'

    kanji_to_arabic = {
        '一': 1, '二': 2, '三': 3, '四': 4, '五': 5,
        '六': 6, '七': 7, '八': 8, '九': 9, '十': 10
    }

    match = re.search(pattern, text)
    if match:
        return int(match.group())
    else:
        numeric_age = 0
        chars = list(text)
        i = 0
        while i < len(chars):
            char = chars[i]
            if char in kanji_to_arabic:
                if char == '十':
                    if i == 0:
                        numeric_age += 10
                    else:
                        numeric_age += kanji_to_arabic[char]
                else:
                    numeric_age += kanji_to_arabic[char]
            i += 1
        return numeric_age

# DurationOfPitch 속성 변형
def convert_duration(item):
    if isinstance(item, str) and item:
        if '秒' in item:
            seconds = int(item.replace('秒', ''))
            return seconds / 60  # 초를 분으로 변환
        elif '分' in item:
            return int(item.replace('分', ''))  # 분 그대로 반환
    return pd.NA

# Gender 속성 변형
def normalize_Str(gender):
    if isinstance(gender, str):
        normalized = unicodedata.normalize('NFKC', gender).lower().strip()  # 공백 제거
        return normalized.replace(" ", "")  # 모든 공백 제거
    return gender

def convert_monthly_income(income):
    if isinstance(income, str) and '月収' in income and '万円' in income:
        # 정규 표현식으로 숫자 추출
        match = re.search(r'(\d+(\.\d+)?)', income)
        if match:
          # 숫자 변환 및 만엔 단위 변환
          return float(match.group(0)) * 10000  # 만엔을 원으로 변환
    return income

# NumberOfTrips 속성 변형 함수
def extract_number_of_trips(trips):
    if isinstance(trips, str):
        match = re.search(r'\d+', trips)  # 숫자 추출
        if match:
            return int(match.group())  # 숫자를 정수로 변환하여 반환
    return trips  # 숫자가 없으면 원래 값 반환

def normalize_word(word):
    if isinstance(word, str):
        # NFKC 정규화
        normalized = unicodedata.normalize('NFKC', word)
        # 알파벳과 숫자만 남기고 나머지는 공백으로 대체
        cleaned = re.sub(r'[^a-zA-Z0-9]', '', normalized)
        return cleaned.lower()  # 소문자로 변환
    return word

tr['Age'] = tr['Age'].apply(lambda age_text: extract_age(age_text) if isinstance(age_text, str) and age_text else None)
tr['DurationOfPitch'] = tr['DurationOfPitch'].apply(convert_duration)
tr['Gender'] = tr['Gender'].apply(normalize_Str)
tr['MonthlyIncome'] = tr['MonthlyIncome'].apply(convert_monthly_income)
tr['NumberOfTrips'] = tr['NumberOfTrips'].apply(lambda trips: 2 if trips == '半年に1回' else trips)
tr['NumberOfTrips'] = tr['NumberOfTrips'].apply(lambda trips: 1 if trips == '四半期に1回' else trips)
tr['NumberOfTrips'] = tr['NumberOfTrips'].apply(extract_number_of_trips)

In [112]:
tr=pd.read_csv('/content/preprocessed_train.csv')

In [113]:
for column in tr.columns:
    if tr[column].isnull().any():  # 널값이 있는 경우에만
        mode_value = tr[column].mode()[0]  # 최빈값 계산
        tr[column].fillna(mode_value, inplace=True)  # 널값을 최빈값으로 채움

In [116]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      3489 non-null   int64  
 1   Age                     3489 non-null   float64
 2   TypeofContact           3489 non-null   object 
 3   CityTier                3489 non-null   int64  
 4   DurationOfPitch         3489 non-null   float64
 5   Occupation              3489 non-null   object 
 6   Gender                  3489 non-null   object 
 7   NumberOfPersonVisiting  3489 non-null   int64  
 8   NumberOfFollowups       3489 non-null   float64
 9   ProductPitched          3489 non-null   object 
 10  PreferredPropertyStar   3489 non-null   int64  
 11  NumberOfTrips           3489 non-null   float64
 12  Passport                3489 non-null   int64  
 13  PitchSatisfactionScore  3489 non-null   int64  
 14  Designation             3489 non-null   

In [130]:
df_corr=tr_g.corr()
df_corr_sort=df_corr.sort_values('ProdTaken', ascending=False)
df_corr_sort['ProdTaken'].tail(10)

,ProdTaken
Designation_senior manager,-0.090884
Occupation_Salaried,-0.094632
Gender_female,-0.095074
Marital_Status_married,-0.102501
NumberOfPersonVisiting,-0.113046
Has_Children,-0.123284
Age,-0.177264
Designation_manager,-0.200278
MonthlyIncome,-0.201250
ProductPitched_deluxe,-0.202652


In [137]:
sns.pairplot(tr_g[tr_g.columns])
plt.show();

KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x7e802b888ca0> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7e802b87beb0> (for post_execute):


KeyboardInterrupt: 

In [117]:
tr_g=pd.get_dummies(tr)

In [118]:
tr_g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3489 entries, 0 to 3488
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             3489 non-null   int64  
 1   Age                            3489 non-null   float64
 2   CityTier                       3489 non-null   int64  
 3   DurationOfPitch                3489 non-null   float64
 4   NumberOfPersonVisiting         3489 non-null   int64  
 5   NumberOfFollowups              3489 non-null   float64
 6   PreferredPropertyStar          3489 non-null   int64  
 7   NumberOfTrips                  3489 non-null   float64
 8   Passport                       3489 non-null   int64  
 9   PitchSatisfactionScore         3489 non-null   int64  
 10  MonthlyIncome                  3489 non-null   float64
 11  Car_Ownership                  3489 non-null   int64  
 12  Has_Children                   3489 non-null   i

In [119]:
prod_taken=tr_g.pop('ProdTaken')

In [120]:
tr_g['ProdTaken'] = prod_taken

In [121]:
X = tr_g.iloc[:,0:33]
Y = tr_g.iloc[:,33]

In [122]:
X=X.astype('float32')
Y=Y.astype('float32')

In [123]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(24,  input_dim=33, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelpath="/content/model"
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# 모델을 실행합니다.
history=model.fit(X, Y, epochs=200, batch_size=10,validation_split=0.25,callbacks=[early_stopping_callback,checkpointer])

Epoch 1/200
262/262 [==============================] - 2s 4ms/step - loss: 858.7255 - accuracy: 0.6678 - val_loss: 0.6532 - val_accuracy: 0.8866
Epoch 2/200
262/262 [==============================] - 1s 3ms/step - loss: 0.6192 - accuracy: 0.8479 - val_loss: 0.5724 - val_accuracy: 0.8866
Epoch 3/200
262/262 [==============================] - 1s 3ms/step - loss: 0.5600 - accuracy: 0.8479 - val_loss: 0.5152 - val_accuracy: 0.8866
Epoch 4/200
262/262 [==============================] - 1s 3ms/step - loss: 0.5184 - accuracy: 0.8479 - val_loss: 0.4731 - val_accuracy: 0.8866
Epoch 5/200
262/262 [==============================] - 1s 3ms/step - loss: 0.4891 - accuracy: 0.8479 - val_loss: 0.4425 - val_accuracy: 0.8866
Epoch 6/200
262/262 [==============================] - 1s 3ms/step - loss: 0.4687 - accuracy: 0.8479 - val_loss: 0.4205 - val_accuracy: 0.8866
Epoch 7/200
262/262 [==============================] - 1s 3ms/step - loss: 0.4544 - accuracy: 0.8479 - val_loss: 0.4040 - val_accuracy: 0.88